In [2]:
using Pkg, SymPy
using BenchmarkTools, LinearAlgebra, LazyArrays, BandedMatrices, BlockArrays

Pkg.status()

      Status `~/.julia/environments/v1.7/Project.toml`
  [aae01518] BandedMatrices v0.17.0
  [6e4b80f9] BenchmarkTools v1.3.1
  [8e7c35d0] BlockArrays v0.16.18
  [ce91de38] ColorBitstring v0.1.1
  [7073ff75] IJulia v1.23.3
  [5078a376] LazyArrays v0.22.10
  [a3b82374] MatrixFactorizations v0.9.1
  [91a5bcdd] Plots v1.29.0
  [438e738f] PyCall v1.93.1
  [295af30f] Revise v3.3.3
  [f8ebbe35] SemiseparableMatrices v0.3.3
  [3cc68bcd] SetRounding v0.2.1
  [24249f21] SymPy v1.1.6
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
  [8dfed614] Test


In [3]:
LinearAlgebra.I

UniformScaling{Bool}
true*I

In [4]:
function backSub(U, b)
    T = eltype(U)
    n = size(U)[1]
    x = zeros(T, n)
    x[n] = b[n] / U[n, n]
    for i in n-1 : -1 : 1
        sum_ = 0.0
        for j in i+1 : n
            sum_ += U[i, j] * x[j]
        end
        x[i] = (b[i] - sum_)/U[i, i]
    end
    x
end

function backSubBanded(U, b, bw=1)
    T = eltype(U)
    n = size(U)[1]
    x = zeros(T, n)
    x[n] = b[n] / U[n, n]
    for i in n-1 : -1 : 1
        sum_ = zero(T)
        for j in i+1 : min(n, i+bw)
            sum_ += U[i, j] * x[j]
        end
        x[i] = (b[i] - sum_) / U[i, i]
    end
    x
end


function backSubBandedmxn(U, B, bw)
    T = eltype(U)
    n = size(U)[2]
    k = size(B)[2]
    X = zeros(T, n, k)

    for j in k : -1 : 1
        b = B[:, j]
        x = zeros(T, n)
        x[n] = b[n] / U[n, n]
        for i in n-1 : -1 : 1
            sum_ = zero(T)
            for j in i+1 : min(n, i+bw)
                sum_ += U[i, j] * x[j]
            end
            x[i] = (b[i] - sum_) / U[i, i]
        end
        X[:, j] = x
    end
    X
end


n = 100
k = 10
bw = 99
U = big.(rand(n, n) .+ 1)
U = triu(U) - triu(U, bw+1)
B = big.(rand(n, k))

X = backSubBandedmxn(U, B, bw)
X
U * X ≈ B

true

In [5]:
T = eltype(A)
S = size(A)

function invBiU(U)
    T = eltype(U)
    n = size(U)[1]
    e_n = zeros(T, n)
    e_n[n] = big(1.0)
    # x = U \ e_n
    x = backSubBanded(U, e_n, 1)
    y = 1 ./ x
    Uinv = triu(x * y')
end
# one(T, S)

invBiU (generic function with 1 method)

In [6]:
function invU_1_0(U)
    T = eltype(U)
    n = size(U)[2]
    e_n = zeros(T, n)
    e_n[n] = one(T)
    x = U \ e_n
    # x = backSubBanded(U, e_n, 1)
    y = inv.(U[diagind(U)] .* x)
    Uinv = triu(x * y')
    Uinv
end


function invU_1_01(U)
    T = eltype(U)
    n = size(U)[2]
    e_n = zeros(T, n)
    e_n[n] = one(T)
    # x = U \ e_n
    x = backSubBanded(U, e_n, 1)
    y = inv.(U[diagind(U)] .* x)
    # y = 1 ./ (U[diagind(U)] .* x)
    Uinv = triu(x * y')
    Uinv
end

# @btime invU_1_0(A);
# @btime invU_1_01(A);
# 898.500 μs (50330 allocations: 2.84 MiB)
# 674.250 μs (31306 allocations: 1.83 MiB)


@time invU_1_0(A);
@time invU_1_01(A);
@time inv(A);

  0.893536 seconds (6.67 M allocations: 339.772 MiB, 7.92% gc time, 93.78% compilation time)
  0.027737 seconds (75.12 k allocations: 4.112 MiB, 97.48% compilation time)
  0.262873 seconds (5.63 M allocations: 300.259 MiB, 14.24% gc time, 38.78% compilation time)


In [13]:
n = 100
bw = 1

A = big.(rand(n, n) .+ 1)
A = triu(A) - triu(A, 2)
A += I

# A[diagind(A)] .= 1

inv(A) ≈ invU_1_0(A)


A = A'

inv(A) ≈ (invU_1_0(A'))'


# B = big.(BandedMatrix(rand(n, n), (0, bw)))
# B += I

# # inv(B)

# invU_1_01(B) * B ≈ I

true

In [143]:
n = 100
bw = 1
U = BandedMatrix(1 .+ big.(rand(n, n)), (0, bw))

for i in 1:n
    U[i, i] = big(1.0)
end
# U = AbstractArray(U)
En = [zeros(eltype(U), (99,)) ; 1]

# U \ En
backSubBanded(U, En, 1)
# ≈ backSubBanded(U, big.([zeros(99, 1) ; 1]), 1)
# eltype(U)


100-element Vector{BigFloat}:
  -1.044839476606806319164283850502569551487821229949895535613613667939057315162901e+15
   5.874772427353937050751401983632501430542690876143450065166103641372896748547595e+14
  -4.518653338433662632789855644952173499120543902793181876552676618350664991378615e+14
   3.548893960193466701169273581202670403683465526827935904601257161248978954438374e+14
  -3.215658535264953407378581891946249468340116250012299268417724690706505698589937e+14
   1.853757473293520754159000156819299274594376091835702070514245355209286809895394e+14
  -1.72193448053071483707237973631568139587768745848930027868639817644883756956632e+14
   1.330980003059845712645666530966298712267906329914259113663893719873821976474831e+14
  -1.188859141340502580238713225546548126009645540805258753273938760800956902562748e+14
   8.544167746887435781459966555004945752122697912341849345744013741536123170613426e+13
  -5.256680164603209999432771516583422758614146534605940641150898109733053148004739e+13
   

In [145]:
invBiU(U) * U ≈ I

true

In [120]:
# l,u = 2,1          # block bandwidths
# N = M = 4          # number of row/column blocks
# cols = rows = 1:N  # block sizes

# BlockBandedMatrix(Zeros(sum(rows),sum(cols)), rows,cols, (l,u)) # creates a block-banded matrix of zeros
# BlockBandedMatrix(Ones(sum(rows),sum(cols)), rows,cols, (l,u)) # creates a block-banded matrix with ones in the non-zero entries
# BlockBandedMatrix(I, rows,cols, (l,u))                          # creates a block-banded  identity matrix